In [6]:
# Importing required libraries
import urllib.request                   # Library to fetch webpages
from bs4 import BeautifulSoup           # Library to parse data from webpages
from time import sleep
!pip install selenium
from selenium import webdriver 
import time
# consist of all the schema of the data we are extracting
from schema import SCHEMA
import pandas as pd
import selenium
import numpy as np

In [7]:
# Setting login details in Glassdoor
login = {
    "username": "kanevi2020@sweatmail.com",
    "password": "testpass123"
}

In [8]:
SCHEMA

['date',
 'employee_title',
 'location',
 'employee_status',
 'review_title',
 'pros',
 'cons',
 'rating_overall']

In [9]:
# accessing chrome webdriver
def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
#     install chromedriver in python directory and add path here
    browser = webdriver.Chrome(options=chrome_options,executable_path=r"D:\\Study\\Text Analytics\\glassdoor-review-scraper-master\\glassdoor-review-scraper-master\\chromedriver")
    return browser
browser = get_browser()

In [10]:
# login to glassdoor webiste using login credential 
def login():
    print('loggging to', login["username"])
    url = 'https://www.glassdoor.com/profile/login_input.htm'
    
    browser.get(url)

    time.sleep(1)                           # Give Javascript time to render
    email_field = browser.find_element_by_id('userEmail')
    time.sleep(1)
    password_field = browser.find_element_by_id('userPassword')
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys(login["username"])
    password_field.send_keys(login["password"])
    submit_btn.click()

In [11]:
res = pd.DataFrame([], columns=SCHEMA)

#### provide the link of the webpage from which we data need to be extracted

In [13]:
amazon_url = 'https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036.htm'
browser.get(amazon_url)

In [14]:
# extracts data from the mentioned class
def extract_from_page():

    def is_featured(review):
        try:
            review.find_element_by_class_name('featuredFlag')
            return True
        except selenium.common.exceptions.NoSuchElementException:
            return False
        
# extracts author inforamtion - employee status, location , Job role
    def extract_review(review):
        author = review.find_element_by_class_name('authorInfo')
        res = {}
        # import pdb;pdb.set_trace()
        for field in SCHEMA:
            res[field] = scrape(field, review, author)

        assert set(res.keys()) == set(SCHEMA)
        return res

    res = pd.DataFrame([], columns=SCHEMA)

    reviews = browser.find_elements_by_class_name('empReview')

    for review in reviews:
        if not is_featured(review):
            data = extract_review(review)
            res.loc[idx[0]] = data
        else:
            print('Discarding a featured review')
        idx[0] = idx[0] + 1
    return res


In [15]:
# checking if there are more than one page for the company
def more_pages():
    try:
        next_ = browser.find_element_by_class_name('pagination__PaginationStyle__next')
        next_.find_element_by_tag_name('a')
        return True
    except selenium.common.exceptions.NoSuchElementException:
        return False
# extracting information in the next page using pagination tag in the page 
def go_to_next_page():
    next_ = browser.find_element_by_class_name(
        'pagination__PaginationStyle__next').find_element_by_tag_name('a')
    browser.get(next_.get_attribute('href'))
    time.sleep(1)
    page[0] = page[0] + 1

In [16]:
# Function to scrape data for each column
def scrape(field, review, author):
# extracts date and time 
    def scrape_date(review):
        return review.find_element_by_tag_name(
            'time').get_attribute('datetime')

# extracts Employee title
    def scrape_emp_title(review):
        if 'Anonymous Employee' not in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorJobTitle').text.split('-')[1]
            except Exception:
                res = np.nan
        else:
            res = np.nan
        return res

# extracts Employee location
    def scrape_location(review):
        if 'in' in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorLocation').text
            except Exception:
                res = np.nan
        else:
            res = np.nan
        return res
    
# extracts Status 
    def scrape_status(review):
        try:
            res = author.text.split('-')[0]
        except Exception:
            res = np.nan
        return res
    
# Extract Review Title
    def scrape_rev_title(review):
        return review.find_element_by_class_name('summary').text.strip('"')

# Get all reviews 
    def expand_show_more(section):
        try:
            more_link = section.find_element_by_class_name('v2__EIReviewDetailsV2__continueReading')
            more_link.click()
        except Exception:
            pass

# Extract pros
    def scrape_pros(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[0].find_elements_by_tag_name('p')[1].text
            #res = res.strip()
        except Exception:
            res = np.nan
        return res

# Extract Cons 
    def scrape_cons(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[1].find_elements_by_tag_name('p')[1].text
        except Exception:
            res = np.nan
        return res

# Extract Overall rating
    def scrape_overall_rating(review):
        try:
            ratings = review.find_element_by_class_name('gdStars')
            overall = ratings.find_element_by_class_name(
                'rating').find_element_by_class_name('value-title')
            res = overall.get_attribute('title')
        except Exception:
            res = np.nan
        return res

    funcs = [
        scrape_date,
        scrape_emp_title,
        scrape_location,
        scrape_status,
        scrape_rev_title,
        scrape_pros,
        scrape_cons,
        scrape_overall_rating,
    ]
    
    fdict = dict((s, f) for (s, f) in zip(SCHEMA, funcs))

    return fdict[field](review)

In [17]:
page = [1]
idx = [0]
res = pd.DataFrame()
reviews_df = extract_from_page()
res = res.append(reviews_df)
date_limit_reached = [False]

Discarding a featured review


In [18]:
# give the number of review required as length of results
while more_pages() and\
        len(res) < 20:
    go_to_next_page()
    reviews_df = extract_from_page()
    res = res.append(reviews_df)
    time.sleep(1)

end = time.time()

In [19]:
res

,date,employee_title,location,employee_status,review_title,pros,cons,rating_overall
1,Mon Mar 02 2020 22:02:02 GMT-0500 (Eastern Sta...,Picker/Packer,"Campbellsville, KY",Former Employee,It was great!,"Incentives, good pay, significant breaks, good...",Not really any complaints at all.,5.0
2,Sun Jan 10 2016 17:33:15 GMT-0500 (Eastern Sta...,NaN,"Seattle, WA",Current Employee,You Get What You Put In,"Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...,5.0
3,Sat Feb 20 2016 18:05:10 GMT-0500 (Eastern Sta...,Senior Engineering Manager,"Seattle, WA",Current Employee,"Exciting Work, Abusive Culture","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur...",3.0
4,Mon Dec 03 2018 12:06:16 GMT-0500 (Eastern Sta...,Software Development Manager,"Phoenix, AZ",Current Employee,Amazing company and the most driven and smarte...,- You can learn a lot very quickly in a very s...,"- Can be overwhelming, very steep learning cur...",5.0
5,Fri Feb 23 2018 13:41:40 GMT-0500 (Eastern Sta...,Software Development Manager,"Phoenix, AZ",Current Employee,An Amazing Place to Work,"I've been at Amazon for a month now, and I've ...","No cons, so far - seriously. Like I said, I'm ...",5.0
6,Tue Dec 17 2013 12:31:58 GMT-0500 (Eastern Sta...,Software Development Manager,"Seattle, WA",Current Employee,"Can be amazing for some people, horrible for o...",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...,4.0
7,Thu Mar 23 2017 08:51:55 GMT-0400 (Eastern Day...,NaN,"Seattle, WA",Former Employee,We are now in a world where we are condescende...,This company gets A list performance from C li...,Maybe because the company recruits C listers w...,4.0
8,Wed Apr 08 2020 14:59:54 GMT-0400 (Eastern Day...,Software Engineer,"Seattle, WA",Current Employee,An Engineer's Dream,Lots of autonomy. Engineering teams drive the ...,Onboarding felt like being thrown into a pool ...,5.0
9,Wed Apr 22 2020 10:20:25 GMT-0400 (Eastern Day...,Senior Solutions Architect,NaN,Current Employee,In Summary: Thoughtful,"Everything from the interview process, to the ...",The organization is very flat. This is actuall...,5.0
10,Fri Apr 24 2020 08:43:55 GMT-0400 (Eastern Day...,BDM,"Seattle, WA",Current Employee,Amazing Place to work,AWS is one of the fastest growing companies on...,"You will be working hard, but when you make it...",5.0
